## 作業

1. 試著調整 RandomForestClassifier(...) 中的參數，並觀察是否會改變結果？
2. 改用其他資料集 (boston, wine)，並與回歸模型與決策樹的結果進行比較

In [1]:
from sklearn import datasets, metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

wine = datasets.load_wine()
x_train, x_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.2, random_state=4)

In [2]:
reg = RandomForestRegressor()
reg.fit(x_train, y_train)
pred = reg.predict(x_test)
acc = metrics.mean_absolute_error(y_test, pred)
print("MAE: ", acc)

MAE:  0.10833333333333334


In [3]:
print("Feature importance: ", reg.feature_importances_)

Feature importance:  [0.0247257  0.00284606 0.00067826 0.0087856  0.0145974  0.
 0.37499049 0.         0.00087874 0.11381767 0.02268658 0.19825759
 0.23773591]


### GridSearchCV暴力試參數

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

def get_best_model_and_accuracy(model, params, X, y):
    # n_jobs=-1 會使用cpu全部的運算力
    grid = GridSearchCV(model, params, cv=5, error_score=0, n_jobs=-1)
    grid.fit(X, y)
    print("Best accuracy: {}".format(grid.best_score_))
    print("Best params: {}".format(grid.best_params_))
    print("Average time to fit (s): {:.3f}".format(grid.cv_results_['mean_fit_time'].mean()))
    print("Average time to score (s): {:.3f}".format(grid.cv_results_['mean_score_time'].mean()))

In [5]:
clf = RandomForestClassifier()

pipeline = Pipeline([
    ('clf', clf)
])
params = {
    'clf__n_estimators': [10, 20, 50, 100],
    'clf__criterion': ['gini', 'entropy'],
    'clf__min_samples_split': [2, 5, 10],
    'clf__min_samples_leaf': [2, 5, 10],
}
get_best_model_and_accuracy(pipeline, params, wine.data, wine.target)

Best accuracy: 0.9775280898876404
Best params: {'clf__criterion': 'gini', 'clf__min_samples_split': 5, 'clf__n_estimators': 20, 'clf__min_samples_leaf': 2}
Average time to fit (s): 0.082
Average time to score (s): 0.004


### 測試找出的最佳參數

In [9]:
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier(n_estimators=50, criterion='gini', min_samples_leaf=5, min_samples_split=10)
rf.fit(x_train, y_train)
y_pred = rf.predict(x_test)

print("Accuracy = {:.3f}".format(accuracy_score(y_test, y_pred)))

Accuracy = 1.000


### 測試深度不同的Tree對準確度造成的影響

In [10]:
for depth in [1, 2, 5, 10]:
    rf = RandomForestClassifier(n_estimators=50, criterion='gini', min_samples_leaf=5, min_samples_split=10, max_depth=depth)
    rf.fit(x_train, y_train)
    y_pred = rf.predict(x_test)

    print("max_depth={}, Accuracy = {:.3f}".format(depth, accuracy_score(y_test, y_pred)))

max_depth=1, Accuracy = 1.000
max_depth=2, Accuracy = 0.972
max_depth=5, Accuracy = 1.000
max_depth=10, Accuracy = 1.000
